# Домашнее задание №2

Выполнил: Тихонов Сергей (БЭК-181)

## Часть 1. ML workflow (**всего 5 баллов**)

In [209]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product
import warnings
warnings.filterwarnings("ignore")

### Загрузим данные для работы. 

In [179]:
df = pd.read_csv("winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Будем решать задачу регрессии: необходимо предсказать качество вина на основе его характеристик

### Шаг 1.  (**0.2 балла**)
Создайте матрицу X объект-признак и целевой вектор y ("quality")

In [180]:
X = df.iloc[:,:-1]
y = df["quality"]

In [181]:
X

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


In [182]:
y

0       5
1       5
2       5
3       6
4       5
       ..
1594    5
1595    6
1596    6
1597    5
1598    6
Name: quality, Length: 1599, dtype: int64

### Шаг 2. (**0.2 балла**)
Разбейте данные на train и test (доля тестовых данных - 30%).

In [659]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)

### Шаг 3. (**0.2 балла**)
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [660]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_pred_train = linreg.predict(X_train)
y_pred_test = linreg.predict(X_test)

### Шаг 4. (**0.4 балла**)
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [661]:
print("Train MSE: {}".format(mean_squared_error(y_train, y_pred_train)))
print("Test MSE: {}".format(mean_squared_error(y_test, y_pred_test)))

print("Train R2: {}".format(r2_score(y_train, y_pred_train)))
print("Test R2: {}".format(r2_score(y_test, y_pred_test)))

Train MSE: 0.43236584075519646
Test MSE: 0.385200371314518
Train R2: 0.3698461447282667
Test R2: 0.32609824481483884


### Шаг 5. (**0.5 балла**)
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [662]:
from sklearn.model_selection import cross_val_score

print(cross_val_score(linreg, X, y, cv=5, scoring='r2').mean())

0.2900416288421967


### Шаг 6.  (**0.5 балла**)
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1. 

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [162]:
from sklearn.linear_model import Lasso

for alpha in np.arange(0.1, 1.1, 0.1):
    
    lasso = Lasso(alpha = alpha)
    lasso.fit(X_train, y_train)

    y_pred_train = lasso.predict(X_train)
    print("Alpha: {}".format(alpha))
    print("R2: {}".format(r2_score(y_train, y_pred_train)))

Alpha: 0.1
R2: 0.21210421978505278
Alpha: 0.2
R2: 0.1528190469973908
Alpha: 0.30000000000000004
R2: 0.07869160541193321
Alpha: 0.4
R2: 0.032051139940901474
Alpha: 0.5
R2: 0.03192165218878873
Alpha: 0.6
R2: 0.03176338938065082
Alpha: 0.7000000000000001
R2: 0.031576351516487966
Alpha: 0.8
R2: 0.03136053859630006
Alpha: 0.9
R2: 0.031115950620086874
Alpha: 1.0
R2: 0.030842587587848858


### Шаг 7.  (**0.5 балла**)
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [166]:
from sklearn.model_selection import GridSearchCV

alphas = np.linspace(1e-10, 5, 200)

params = {'alpha':alphas}
cv = GridSearchCV(lasso,
                  params,
                  scoring='r2',
                  cv=5
                 )
cv.fit(X_train, y_train)

print("Лучший алгорим: \n{}".format(cv.best_estimator_))
print("Лучший R2: {}".format(cv.best_score_))

Лучший алгорим: 
Lasso(alpha=1e-10, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
Лучший R2: 0.32452139399189867


### Шаг 8.  (**0.5 балла**)

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [167]:
best_predict = cv.best_estimator_.predict(X_test)
print("Лучший R2: {}".format(r2_score(y_test, best_predict)))

Лучший R2: 0.37440421639821064


### Шаг 9.  (**0.5 балла**)

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [172]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

p1 = Pipeline([
    ('poly', PolynomialFeatures(2)),
    ('model', LinearRegression())
    ])

In [173]:
print(cross_val_score(p1, X, y, cv=5, scoring='r2').mean())

0.23009616556632376


### Шаг 10.  (**0.5 балла**)
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [177]:
p1.fit(X_train, y_train)
y_pred_train = p1.predict(X_train)
y_pred_test = p1.predict(X_test)
print("Train MSE: {}".format(mean_squared_error(y_train, y_pred_train)))
print("Test MSE: {}".format(mean_squared_error(y_test, y_pred_test)))

print("Train R2: {}".format(r2_score(y_train, y_pred_train)))
print("Test R2: {}".format(r2_score(y_test, y_pred_test)))

Train MSE: 0.348112006632927
Test MSE: 0.44985397524710785
Train R2: 0.4488961565705125
Test R2: 0.35581769654352113


### Сделайте выводы. Для этого ответьте на вопросы: (**1 балл**)

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

#### Ответ:

1) Исходная модель (**Линейная регрессия** без регуляризации) показала низкое качество по метрикам $R^{2}$ и MSE. Возможная причина такого результата - переобучение (модель сильно подстраивается под тренировочные данные, а на новых данных показывает низкий результат). Качество на **test** (cross-validation): 
$$R^{2} = 0.2900416288421967$$

2) Использование $L_1$ регуляризации повысило $R^{2}$ на **test**, что подтверждает наличие переобучения в модели. Качество на **test** (cross-validation):
$$R^{2} = 0.37440421639821064$$

3) Качество модели на **train** выросло значительно, при этом качество модели на **test** немного снизилось. Таким образом, можно говорить о переобучении модели. Качество на **test** могло не увеличится из-за того, что истинная зависимость между предсказываемой и предсказывающими переменными не описывается степенными функциями. Важно отметить, что $R^{2}$ при добавлении новых признаков в модель всегда увеличивается, поэтому стоит отказаться от использования его в случае добавления полиномиальных признаков. 

### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы.

При улучшении качества r2 на 0.1-0.2 +1 балл, при большем улучшении +2 балла (дополнительно к 5 баллам за основную часть).

In [1301]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=121)

Попробуем то, чего ещё не было использовано в домашке:

* StandardScaler

* GBRegressor

In [1334]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor

np.random.seed(1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

gb = GradientBoostingRegressor(learning_rate=0.0885, alpha=0.7, n_estimators=99, max_depth=3)
gb.fit(X_train, y_train)

y_pred = gb.predict(X_test)

In [1335]:
print("Test MSE: {}".format(mean_squared_error(y_test, y_pred)))
print("Test R2: {}".format(r2_score(y_test, y_pred)))

Test MSE: 0.3533893325576615
Test R2: 0.4751643575876314


Итог: удалось достичь более высокого качества!

## Часть 2. Target encoding (**всего 5 баллов**)

В этом части домашнего задания вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подготовка данных

In [203]:
sales = pd.read_csv('sales_train.csv.gz')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [204]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [205]:
all_data

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0
...,...,...,...,...
10768834,59,22162,33,0.0
10769024,59,22163,33,0.0
10769690,59,22164,33,0.0
10771216,59,22166,33,0.0


### Mean encodings без регуляризации

После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`. 

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [206]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621699


#### Способ 2

In [207]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621699


###  KFold схема (**1.25 балла**)

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection. 

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях. 

In [211]:
# YOUR CODE GOES HERE
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=False)

for train_index, val_index in kf.split(all_data):
        item_id_target_mean = all_data.loc[train_index].groupby('item_id').target.mean()
        all_data['item_target_enc'].loc[val_index] = all_data['item_id'].map(item_id_target_mean)

all_data['item_target_enc'].fillna(0.3, inplace=True) 
encoded_feature = all_data['item_target_enc'].values

# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.416599907917136


Ожидаемый ответ 0.4165

### Leave-one-out схема (**1.25 балла**)

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`. 

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

In [202]:
# YOUR CODE GOES HERE
all_data['sum'] = all_data.groupby('item_id')['target'].transform('sum')
all_data['count'] = all_data.groupby('item_id')['target'].transform('count')
all_data['item_target_enc'] = (all_data['sum'] - all_data['target']) / (all_data['count'] - 1)

encoded_feature = all_data['item_target_enc'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4803848311293002


Ожидаемый ответ 0.4803

### Smoothing (**1.25 балла**)

Необходимо реализовать smoothing с $\alpha = 100$. Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конктертной категории, а не количество строк в датасете.

In [331]:
# YOUR CODE GOES HERE

target_mean = all_data.groupby('item_id').target.mean()
nrows = all_data.groupby('item_id').target.count()
smoothing_target_mean = (target_mean * nrows + 0.3343 * 100) / (nrows + 100)
all_data['item_target_enc'] = all_data['item_id'].map(smoothing_target_mean)
encoded_feature = all_data['item_target_enc'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4818198797097264


Ожидаемый ответ 0.4818

### Expanding mean схема (**1.25 балла**)

Необходимо реализовать *expanding mean* схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

**Комменатрий к коду**:

Добавляю 1 в cumcount, поскольку счёт идёт от "0" до "количества элементов - 1", поэтому без добавления единицы будет происходить деление на 0.

In [842]:
# YOUR CODE GOES HERE

cumsum = all_data.groupby('item_id').target.cumsum() 
cumcount = all_data.groupby('item_id').target.cumcount() 
all_data['item_target_enc'] = (cumsum - all_data.target) / cumcount 

all_data['item_target_enc'].fillna(0.3343, inplace=True) 
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.5025245211081697


Ожидаемый ответ 0.5025